In [18]:
import numpy as np
import keras
import tensorflow as tf
from keras import layers

max_features = 20000  
maxlen = 200  

In [19]:

inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(max_features, 128)(inputs)
x = layers.LSTM(64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_4 (Embedding)     (None, None, 128)         2560000   
                                                                 
 lstm_8 (LSTM)               (None, None, 64)          49408     
                                                                 
 lstm_9 (LSTM)               (None, 64)                33024     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,642,497
Trainable params: 2,642,497
Non-trainable params: 0
_________________________________________________________________


In [20]:
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.imdb.load_data(
    num_words=max_features
)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")

x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

25000 Training sequences
25000 Validation sequences


In [21]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val))

Epoch 1/3
782/782 [==============================] - 150s 188ms/step - loss: 0.3821 - accuracy: 0.8290 - val_loss: 0.3326 - val_accuracy: 0.8664
Epoch 2/3
782/782 [==============================] - 145s 185ms/step - loss: 0.2080 - accuracy: 0.9223 - val_loss: 0.3289 - val_accuracy: 0.8637
Epoch 3/3
782/782 [==============================] - 149s 191ms/step - loss: 0.1258 - accuracy: 0.9544 - val_loss: 0.4025 - val_accuracy: 0.8520


In [22]:

inputs = keras.Input(shape=(None,), dtype="int32")
x = layers.Embedding(max_features, 128)(inputs)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_5 (Embedding)     (None, None, 128)         2560000   
                                                                 
 bidirectional_2 (Bidirectio  (None, None, 128)        98816     
 nal)                                                            
                                                                 
 bidirectional_3 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 2,757,761
Trainable params: 2,757,761
Non-tra

In [24]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val))

Epoch 1/3
782/782 [==============================] - 269s 337ms/step - loss: 0.3982 - accuracy: 0.8179 - val_loss: 0.3588 - val_accuracy: 0.8512
Epoch 2/3
782/782 [==============================] - 437s 559ms/step - loss: 0.2158 - accuracy: 0.9172 - val_loss: 0.3467 - val_accuracy: 0.8542
Epoch 3/3
782/782 [==============================] - 604s 773ms/step - loss: 0.1295 - accuracy: 0.9554 - val_loss: 0.3630 - val_accuracy: 0.8597


In [25]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
%matplotlib inline
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional

In [26]:
df = pd.read_csv('https://raw.githubusercontent.com/kenneth-lee-ch/SMS-Spam-Classification/master/spam.csv', encoding='ISO-8859-1')
df = df[['v1','v2']]
df.rename(columns={'v1':'label', 'v2':'message'}, inplace=True)
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [27]:
df.describe()

,label,message
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [28]:

df['text_length'] = df['message'].apply(len)
df['msg_type'] = df['label'].map({'ham':0, 'spam':1})
label = df['msg_type'].values
df.head()

,label,message,text_length,msg_type
0,ham,"Go until jurong point, crazy.. Available only ...",111,0
1,ham,Ok lar... Joking wif u oni...,29,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,1
3,ham,U dun say so early hor... U c already then say...,49,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,0


In [29]:
x_train, x_test, y_train, y_test = train_test_split(df['message'], label, test_size=0.2, random_state=434)

In [30]:

max_len = 50 
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
vocab_size = 500

In [31]:
tokenizer = Tokenizer(num_words = vocab_size, 
                      char_level = False,
                      oov_token = oov_tok)
tokenizer.fit_on_texts(x_train)

In [32]:

word_index = tokenizer.word_index
total_words = len(word_index)
total_words

8004

In [33]:
training_sequences = tokenizer.texts_to_sequences(x_train)
training_padded = pad_sequences(training_sequences,
                                maxlen = max_len,
                                padding = padding_type,
                                truncating = trunc_type)

In [34]:
testing_sequences = tokenizer.texts_to_sequences(x_test)
testing_padded = pad_sequences(testing_sequences,
                               maxlen = max_len,
                               padding = padding_type,
                               truncating = trunc_type)

In [35]:
print('Shape of training tensor: ', training_padded.shape)
print('Shape of testing tensor: ', testing_padded.shape)

Shape of training tensor:  (4457, 50)
Shape of testing tensor:  (1115, 50)


In [37]:
# Define parameter
vocab_size = 500 
embedding_dim = 16
drop_value = 0.2
n_dense = 24
n_lstm = 128
drop_lstm = 0.2 
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model1.add(SpatialDropout1D(drop_lstm))
model1.add(LSTM(n_lstm, return_sequences=False))
model1.add(Dropout(drop_lstm))
model1.add(Dense(1, activation='sigmoid'))

In [38]:
model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 50, 16)            8000      
                                                                 
 spatial_dropout1d (SpatialD  (None, 50, 16)           0         
 ropout1D)                                                       
                                                                 
 lstm_12 (LSTM)              (None, 128)               74240     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 82,369
Trainable params: 82,369
Non-trainable params: 0
__________________________________________________

In [39]:
model1.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics = ['accuracy'])

In [40]:
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=4)
history = model1.fit(training_padded,
                     y_train,
                     epochs=num_epochs, 
                     validation_data=(testing_padded, y_test),
                     callbacks =[early_stop],
                     verbose=2)

Epoch 1/30
140/140 - 11s - loss: 0.2384 - accuracy: 0.9152 - val_loss: 0.1259 - val_accuracy: 0.9686 - 11s/epoch - 75ms/step
Epoch 2/30
140/140 - 8s - loss: 0.0744 - accuracy: 0.9798 - val_loss: 0.0833 - val_accuracy: 0.9731 - 8s/epoch - 57ms/step
Epoch 3/30
140/140 - 8s - loss: 0.0875 - accuracy: 0.9776 - val_loss: 0.1523 - val_accuracy: 0.9614 - 8s/epoch - 57ms/step
Epoch 4/30
140/140 - 8s - loss: 0.0588 - accuracy: 0.9870 - val_loss: 0.1007 - val_accuracy: 0.9731 - 8s/epoch - 57ms/step
Epoch 5/30
140/140 - 8s - loss: 0.0426 - accuracy: 0.9901 - val_loss: 0.1091 - val_accuracy: 0.9713 - 8s/epoch - 58ms/step
Epoch 6/30
140/140 - 8s - loss: 0.0367 - accuracy: 0.9919 - val_loss: 0.1183 - val_accuracy: 0.9731 - 8s/epoch - 59ms/step


In [41]:
model2 = Sequential()
model2.add(Embedding(vocab_size,
                     embedding_dim,
                     input_length = max_len))
model2.add(Bidirectional(LSTM(n_lstm,
                              return_sequences = False)))
model2.add(Dropout(drop_lstm))
model2.add(Dense(1, activation='sigmoid'))

In [42]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 50, 16)            8000      
                                                                 
 bidirectional_4 (Bidirectio  (None, 256)              148480    
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_5 (Dense)             (None, 1)                 257       
                                                                 
Total params: 156,737
Trainable params: 156,737
Non-trainable params: 0
_________________________________________________________________


In [43]:
model2.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics=['accuracy'])

In [44]:
num_epochs = 30
early_stop = EarlyStopping(monitor = 'val_loss',
                           patience = 4)
history = model2.fit(training_padded,
                     y_train,
                     epochs = num_epochs,
                     validation_data = (testing_padded, y_test),
                     callbacks = [early_stop],
                     verbose = 2)

Epoch 1/30
140/140 - 13s - loss: 0.2046 - accuracy: 0.9349 - val_loss: 0.0830 - val_accuracy: 0.9731 - 13s/epoch - 92ms/step
Epoch 2/30
140/140 - 9s - loss: 0.0633 - accuracy: 0.9816 - val_loss: 0.0950 - val_accuracy: 0.9749 - 9s/epoch - 64ms/step
Epoch 3/30
140/140 - 9s - loss: 0.0908 - accuracy: 0.9731 - val_loss: 0.1193 - val_accuracy: 0.9632 - 9s/epoch - 65ms/step
Epoch 4/30
140/140 - 9s - loss: 0.0657 - accuracy: 0.9832 - val_loss: 0.1044 - val_accuracy: 0.9740 - 9s/epoch - 65ms/step
Epoch 5/30
140/140 - 9s - loss: 0.0483 - accuracy: 0.9868 - val_loss: 0.0904 - val_accuracy: 0.9776 - 9s/epoch - 65ms/step


In [45]:
model3 = Sequential()
model3.add(Embedding(vocab_size,
                     embedding_dim,
                     input_length = max_len))
model3.add(SpatialDropout1D(0.2))
model3.add(GRU(128, return_sequences = False))
model3.add(Dropout(0.2))
model3.add(Dense(1, activation = 'sigmoid'))
model3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 50, 16)            8000      
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 50, 16)           0         
 lDropout1D)                                                     
                                                                 
 gru (GRU)                   (None, 128)               56064     
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 1)                 129       
                                                                 
Total params: 64,193
Trainable params: 64,193
Non-trainable params: 0
__________________________________________________

In [46]:
model3.compile(loss = 'binary_crossentropy',
                       optimizer = 'adam',
                       metrics=['accuracy'])

In [47]:
num_epochs = 30
early_stop = EarlyStopping(monitor='val_loss', patience=4)
history = model3.fit(training_padded,
                     y_train,
                     epochs=num_epochs, 
                     validation_data=(testing_padded, y_test),
                     callbacks =[early_stop],
                     verbose=2)

Epoch 1/30
140/140 - 6s - loss: 0.4285 - accuracy: 0.8611 - val_loss: 0.3770 - val_accuracy: 0.8771 - 6s/epoch - 41ms/step
Epoch 2/30
140/140 - 3s - loss: 0.3490 - accuracy: 0.8690 - val_loss: 0.1540 - val_accuracy: 0.9462 - 3s/epoch - 24ms/step
Epoch 3/30
140/140 - 3s - loss: 0.0795 - accuracy: 0.9780 - val_loss: 0.0800 - val_accuracy: 0.9749 - 3s/epoch - 24ms/step
Epoch 4/30
140/140 - 3s - loss: 0.0486 - accuracy: 0.9865 - val_loss: 0.0717 - val_accuracy: 0.9812 - 3s/epoch - 24ms/step
Epoch 5/30
140/140 - 3s - loss: 0.0478 - accuracy: 0.9854 - val_loss: 0.0684 - val_accuracy: 0.9794 - 3s/epoch - 25ms/step
Epoch 6/30
140/140 - 3s - loss: 0.0369 - accuracy: 0.9890 - val_loss: 0.0677 - val_accuracy: 0.9821 - 3s/epoch - 24ms/step
Epoch 7/30
140/140 - 3s - loss: 0.0354 - accuracy: 0.9899 - val_loss: 0.0677 - val_accuracy: 0.9812 - 3s/epoch - 24ms/step
Epoch 8/30
140/140 - 3s - loss: 0.0268 - accuracy: 0.9928 - val_loss: 0.0747 - val_accuracy: 0.9803 - 3s/epoch - 25ms/step
Epoch 9/30
140/1

In [48]:

print(f"LSTM model loss and accuracy: {model1.evaluate(testing_padded, y_test)} " )
print(f"Bi-LSTM model loss and accuracy: {model2.evaluate(testing_padded, y_test)} " )
print(f"GRU model loss and accuracy: {model3.evaluate(testing_padded, y_test)}")

35/35 [==============================] - 0s 12ms/step - loss: 0.1183 - accuracy: 0.9731
LSTM model loss and accuracy: [0.11825458705425262, 0.9730941653251648] 
35/35 [==============================] - 1s 16ms/step - loss: 0.0904 - accuracy: 0.9776
Bi-LSTM model loss and accuracy: [0.09035880863666534, 0.9775784611701965] 
35/35 [==============================] - 0s 9ms/step - loss: 0.0875 - accuracy: 0.9812
GRU model loss and accuracy: [0.08745471388101578, 0.9811659455299377]


In [51]:
predict_msg = ["Have friends and colleagues who could benefit from these weekly updates? Send them to this link to subscribe","Call me","Our records show you overpaid for (a product or service). Kindly supply your bank routing and account number to receive your refund."]
def predict_spam(predict_msg):
  new_seq = tokenizer.texts_to_sequences(predict_msg)
  padded = pad_sequences(new_seq,
                         maxlen = max_len,
                         padding = padding_type,
                         truncating = trunc_type)
  return(model1.predict(padded))
predict_spam(predict_msg)

1/1 [==============================] - 0s 331ms/step


array([[0.00958992],
       [0.00429714],
       [0.99173695]], dtype=float32)